# Summary of data in csv files

## Setup and installations

In [0]:
%pip install pandas
%pip install openai
%pip install markdown
%restart_python

In [0]:
import pandas as pd
import openai
import os
import json
import markdown

## Read in data and metadata from the publication of interest

In [0]:
national_char_data = pd.read_csv("data/202324_national_char_data_revised.csv")
national_char_metadata = pd.read_csv("data/202324_national_char_data_revised.meta.csv")

## Create JSONs for each characteristic

In [0]:
# Filter for All state-funded schools' rows only
national_char_data_all_statefunded = national_char_data[national_char_data['establishment_type_group'] == 'All state-funded']

national_char_data_measures = national_char_data_all_statefunded[['time_period', 'sex', 'ethnicity_major', 'ethnicity_minor', 'free_school_meals', 'sen_provision', 'sen_status', 'sen_primary_need', 'disadvantage', 'first_language', 'religious_denomination', 'admission_type', 't_pupils', 'avg_att8', 'pt_l2basics_95', 'avg_p8score', 'pt_ebacc_e_ptq_ee', 'avg_ebaccaps']]

# Create the new 'ethnicity' column
national_char_data_measures['ethnicity'] = national_char_data_measures.apply(
    lambda row: row['ethnicity_minor'] if row['ethnicity_minor'] != 'Total' else row['ethnicity_major'],
    axis=1
)

# Modify the 'ethnicity_major' column to only be not 'Total' if 'ethnicity_minor' is 'Total'
national_char_data_measures['ethnicity_major'] = national_char_data_measures.apply(
    lambda row: row['ethnicity_major'] if row['ethnicity_minor'] == 'Total' else 'Total',
    axis=1
)

# List of columns to iterate over
columns_to_vary = [
    'sex', 'ethnicity_major', 'ethnicity_minor', 'free_school_meals',
    'sen_provision', 'sen_status', 'sen_primary_need',
    'disadvantage','first_language', 'admission_type', 'religious_denomination'
]

# Base filter
base_filter = {
    'sex': 'Total',
    'ethnicity_major': 'Total',
    'ethnicity_minor': 'Total',
    'admission_type': 'Total',
    'sen_provision': 'Total',
    'sen_primary_need': 'Total',
    'sen_status': 'Total',
    'disadvantage': 'Total',
    'free_school_meals': 'Total',
    'first_language': 'Total',
    'religious_denomination': 'Total'
}

# Store results
json_outputs = {}

for col in columns_to_vary:
    # Create a copy of the base filter
    filter_conditions = base_filter.copy()

    # Modify the current column to != 'Total'
    filtered_data = national_char_data_measures[
        pd.concat([
            national_char_data_measures[c] == v if c != col else national_char_data_measures[c] != 'Total'
            for c, v in filter_conditions.items()
        ], axis=1).all(axis=1)
    ]

    # Convert to JSON and store
    json_outputs[col] = filtered_data.to_json(orient='records')

# Convert the metadata to JSON for the prompt
json_metadata_for_prompt = national_char_metadata.to_json(orient="records")


## Query

In [0]:
# Retrieve the API key from Databricks secrets
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = openai.OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="https://adb-2220072380334347.7.azuredatabricks.net/serving-endpoints"
)

# What indicators are the focus?
performance_measures = [
    "pt_ebacc_e_ptq_ee",
    "pt_ebacc_94",
    "pt_ebacc_95",
    "avg_ebaccaps"
]

system_content = "You are creating narrative sections around specific policy areas based on the data provided. The narrative is a national statistics, statistical release so it has to be very matter-of-fact. "

# List of characteristics to loop through
characteristics = [
    'sex', 'ethnicity_major', 'ethnicity_minor',
    'admission_type', 'sen_provision', 'sen_primary_need',
    'sen_status', 'disadvantage', 'free_school_meals',
    'first_language', 'religious_denomination'
]

# Store markdown sections
markdown_sections = []

for characteristic in characteristics:
    # Accordion title
    accordion_title = f"### EBacc entry and achievement by {characteristic.replace('_', ' ').title()}\n"

    # Prompt for the model
    prompt = f"""
Here is the data:
{json_outputs[characteristic]}
Here is the metadata: 
{json_metadata_for_prompt}
The indicators of interest are:
{performance_measures}
Create a summary for {characteristic} featuring tables and a short narrative summary based on the tables. The summary needs to fit under the heading {accordion_title.strip()}. For each indicator of interest available for {characteristic} please create one summary table using the unique values under {characteristic} as rows and time_period as columns. Calculate the gaps in the indicators across the unique values in the {characteristic} column and produce separate tables for gaps (these should only be rendered where there are less than 6 unique values, and no gaps tabled should be rendered for ethnicity_minor or religious_denomination). time_period specifies the academic year (for example 202324 is 2023/24). time_period should be in the order 202324, 202223, 202122, 202021, 201920, 201819. Values should remain with the time_period they relate to. Use the metadata to understand what the indicators are and name the table according to the metadata information on the indicator used. Don't guess what the indicators are, the metadata will tell you. If an indicator of interest is not available at all for {characteristic} do not attempt to produce a table. Tables should follow the order pt_ebacc_e_ptq_ee, pt_ebacc_94 pt_ebacc_95,avg_ebaccaps, then tables for the gaps in each of these indicators in the same order. Do not produce a blank table if an indicator is unavailable. If any indicators do not have complete timeseries, it is okay to just focus on the years where these indicators are available for those indicators. Please provide a short narrative summary based on only the information in the tables. 
"""

    # Send to LLM
    response = client.chat.completions.create(
    model="databricks-meta-llama-3-3-70b-instruct",
    messages=[
        {
            "role": "system",
            "content": system_content
        },
        {
            "role": "user",
            "content": prompt
        }
    ],
    temperature=0.01,
    max_tokens=5000
    )   

    # Append the markdown section
    markdown_sections.append(accordion_title + response.choices[0].message.content + "\n\n")

# Combine all sections into one markdown document
final_markdown = "# EBacc Entry and Achievement Summary\n\n" + "\n".join(markdown_sections)

# Optionally print or save
print(final_markdown)

# To save to a file:
with open("ebacc_summary.md", "w", encoding="utf-8") as f:
    f.write(final_markdown)


## Feed report back into LLM for editing

In [0]:
# Retrieve the API key from Databricks secrets
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = openai.OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="https://adb-2220072380334347.7.azuredatabricks.net/serving-endpoints"
)

# Read the Markdown file (or change to .html if preferred)
with open("ebacc_summary.md", "r", encoding="utf-8") as md_file:
    txt_content = md_file.read()

# Define a new system context
system_content = "You an expert editor who specialises in summarising reports for national statistics."

# Define the prompt
prompt = f"""
Please:
- Remove redundant content across the subsections
- Remove unnecessary information, e.g., about things that are not available
- Do not lose the messages about the characteristics and gaps in indicators in the sub sections.
- Do not lose subsections.
- Do not lose the timeseries information.
- Create a bullet-pointed summary at the top of the report that summarises any key points.
- Improve the clarity and flow across sections.
Here is the report:
{txt_content}
"""

# Send to LLM
# "databricks-meta-llama-3-3-70b-instruct"
# "databricks-llama-4-maverick"
response = client.chat.completions.create(
model="databricks-meta-llama-3-3-70b-instruct",
messages=[
    {
        "role": "system",
        "content": system_content
    },
    {
        "role": "user",
        "content": prompt
    }
],
temperature=0.01,
max_tokens=5000
)   

# Append the markdown section
cleaned_summary = response.choices[0].message.content

# Save the edited report
with open("ebacc_summary_cleaned.md", "w", encoding="utf-8") as f:
    f.write(cleaned_summary)

## Convert .md to a .html

In [0]:

# Read the Markdown file
with open("ebacc_summary_cleaned.md", "r", encoding="utf-8") as f:
    markdown_content = f.read()

# Convert Markdown to HTML
html_content = markdown.markdown(markdown_content, extensions=['tables'])

# Save the HTML content to a file
with open("ebacc_summary_cleaned.html", "w", encoding="utf-8") as f:
    f.write(html_content)
